# Pivot tables

This notebook is my exercise for understanding pivot tables. It is based in part on Chris Moffitt's tutorial in [PBPYTHON](https://pbpython.com/pandas-pivot-table-explained.html).

In [1]:
import datasense as ds
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('sales-funnel.xlsx')

In [4]:
ds.dataframe_info(
 df=df,
 filein='sales-funnel.xlsx')

--------------------------
DataFrame information for: sales-funnel.xlsx

Rows total        : 17
Rows empty        : 0 (deleted)
Rows not empty    : 17
Columns total     : 8
Columns empty     : 0 (deleted)
Columns not empty : 8

Information about non-empty columns
 Column     Data type   Empty cell count   Empty cell percentage 
---------- ----------- ------------------ -----------------------
 Account    int64                      0                     0.0 
 Name       object                     0                     0.0 
 Rep        object                     0                     0.0 
 Manager    object                     0                     0.0 
 Product    object                     0                     0.0 
 Quantity   int64                      0                     0.0 
 Price      int64                      0                     0.0 
 Status     object                     0                     0.0 

List of 8 non-empty columns:
Account, Manager, Name, Price, Product, Quanti

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


In [5]:
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [6]:
# Pivot table using index and values parameters.
# By default the aggfunc is the average.
pd.pivot_table(df, values=['Price'], index=['Manager']).round(2)

,Price
Manager,
Debra Henley,26111.11
Fred Anderson,35875.00


In [7]:
# To be explicit.
pd.pivot_table(df,
               values=['Price'],
               index=['Manager'],
               aggfunc='mean').round(2)

,Price
Manager,
Debra Henley,26111.11
Fred Anderson,35875.00


In [8]:
# The index can have multiple levels.
pd.pivot_table(df,
               values=['Price'],
               index=['Manager', 'Rep'],
               aggfunc='mean').round(2)

Price
Manager       Rep                    
Debra Henley  Craig Booker   20000.00
              Daniel Hilton  38333.33
              John Smith     20000.00
Fred Anderson Cedric Moss    27500.00
              Wendy Yule     44250.00

In [9]:
# The aggfunc can have several parameters.
pd.pivot_table(df,
               values=['Price'],
               index=['Manager', 'Rep'],
               aggfunc=[np.mean, np.sum, len]).round(2)

mean     sum   len
                                Price   Price Price
Manager       Rep                                  
Debra Henley  Craig Booker   20000.00   80000     4
              Daniel Hilton  38333.33  115000     3
              John Smith     20000.00   40000     2
Fred Anderson Cedric Moss    27500.00  110000     4
              Wendy Yule     44250.00  177000     4

In [10]:
# The columns parameter is optional.
# It provides an additional way to segment values.
# The index can have multiple levels.
pd.pivot_table(df,
               values=['Price'],
               index=['Manager', 'Rep'],
               columns=['Product'],
               aggfunc=[np.sum]).round(2)

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

In [11]:
# Replace the NaN with 0.
pd.pivot_table(df,
               values=['Price'],
               index=['Manager', 'Rep'],
               columns=['Product'],
               aggfunc=[np.sum],
               fill_value=0).round(2)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [12]:
# Add another column to the values parameter.
pd.pivot_table(df,
               values=['Price', 'Quantity'],
               index=['Manager', 'Rep'],
               columns=['Product'],
               aggfunc=[np.sum],
               fill_value=0).round(2)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

In [13]:
# Move the product column to the index.
pd.pivot_table(df,
               values=['Price', 'Quantity'],
               index=['Manager', 'Rep', 'Product'],
               aggfunc=[np.sum],
               fill_value=0).round(2)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

In [14]:
# Show totals.
pd.pivot_table(df,
               values=['Price', 'Quantity'],
               index=['Manager', 'Rep', 'Product'],
               aggfunc=[np.sum],
               fill_value=0,
               margins=True).round(2)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2
All                                      522000       30

In [15]:
# Change the categories to look at the Manager level, add Status, remove Quantity.
pd.pivot_table(df,
               values=['Price', 'Quantity'],
               index=['Manager', 'Status'],
               aggfunc=[np.sum],
               fill_value=0,
               margins=True).round(2)

sum         
                          Price Quantity
Manager       Status                    
Debra Henley  declined    70000        2
              pending     50000        6
              presented   50000        3
              won         65000        2
Fred Anderson declined    65000        2
              pending      5000        1
              presented   45000        4
              won        172000       10
All                      522000       30

In [16]:
# Pass a dictionary to the aggfunc to perform different functions.
pd.pivot_table(df,
               values=['Price', 'Quantity'],
               index=['Manager', 'Status'],
               columns=['Product'],
               aggfunc={'Quantity':len, 'Price':np.sum},
               fill_value=0,
               margins=True).round(2)

Price                                      Quantity  \
Product                     CPU Maintenance Monitor Software     All      CPU   
Manager       Status                                                            
Debra Henley  declined    70000           0       0        0   70000        2   
              pending     40000       10000       0        0   50000        1   
              presented   30000           0       0    20000   50000        1   
              won         65000           0       0        0   65000        1   
Fred Anderson declined    65000           0       0        0   65000        1   
              pending         0        5000       0        0    5000        0   
              presented   30000           0    5000    10000   45000        1   
              won        165000        7000       0        0  172000        2   
All                      465000       22000    5000    30000  522000        9   

                                                          
Product                 Maintenance Monitor Software All  
Manager       Status                                      
Debra Henley  declined            0       0        0   2  
              pending             2       0        0   3  
              presented           0       0        2   3  
              won                 0       0        0   1  
Fred Anderson declined            0       0        0   1  
              pending             1       0        0   1  
              presented           0       1        1   3  
              won                 1       0        0   3  
All                               4       1        3  17

In [17]:
# Pass a dictionary to the aggfunc to perform different functions.
# Each value can have a dictionary. Need to remove totals (why?).
pd.pivot_table(df,
               values=['Price', 'Quantity'],
               index=['Manager', 'Status'],
               columns=['Product'],
               aggfunc={'Quantity':len, 'Price':[np.sum, np.mean]},
               fill_value=0).round(2)

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  declined   35000           0       0        0   70000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              won        65000           0       0        0   65000   
Fred Anderson declined   65000           0       0        0   65000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              won        82500        7000       0        0  165000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  declined            0       0        0        2           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              won                 0       0        0        1           0   
Fred Anderson declined            0       0        0        1           0   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              won              7000       0        0        2           1   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  declined        0        0  
              pending         0        0  
              presented       0        2  
              won             0        0  
Fred Anderson declined        0        0  
              pending         0        0  
              presented       1        1  
              won             0        0

In [18]:
# Do this again and save to a dataframe.
table = pd.pivot_table(df,
                       values=['Price', 'Quantity'],
                       index=['Manager', 'Status'],
                       columns=['Product'],
                       aggfunc={'Quantity':len, 'Price':[np.sum, np.mean]},
                       fill_value=0).round(2)

In [19]:
table

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  declined   35000           0       0        0   70000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              won        65000           0       0        0   65000   
Fred Anderson declined   65000           0       0        0   65000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              won        82500        7000       0        0  165000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  declined            0       0        0        2           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              won                 0       0        0        1           0   
Fred Anderson declined            0       0        0        1           0   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              won              7000       0        0        2           1   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  declined        0        0  
              pending         0        0  
              presented       0        2  
              won             0        0  
Fred Anderson declined        0        0  
              pending         0        0  
              presented       1        1  
              won             0        0

In [20]:
table.sort_values(by=('Price', 'mean', 'CPU'), ascending=False)

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Fred Anderson won        82500        7000       0        0  165000   
Debra Henley  won        65000           0       0        0   65000   
Fred Anderson declined   65000           0       0        0   65000   
Debra Henley  pending    40000        5000       0        0   40000   
              declined   35000           0       0        0   70000   
              presented  30000           0       0    10000   30000   
Fred Anderson presented  30000           0    5000    10000   30000   
              pending        0        5000       0        0       0   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Fred Anderson won              7000       0        0        2           1   
Debra Henley  won                 0       0        0        1           0   
Fred Anderson declined            0       0        0        1           0   
Debra Henley  pending         10000       0        0        1           2   
              declined            0       0        0        2           0   
              presented           0       0    20000        1           0   
Fred Anderson presented           0    5000    10000        1           0   
              pending          5000       0        0        0           1   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Fred Anderson won             0        0  
Debra Henley  won             0        0  
Fred Anderson declined        0        0  
Debra Henley  pending         0        0  
              declined        0        0  
              presented       0        2  
Fred Anderson presented       1        1  
              pending         0        0

In [21]:
# Filter the dataframe for one manager
table.query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley declined   35000           0       0        0  70000           0   
             pending    40000        5000       0        0  40000       10000   
             presented  30000           0       0    10000  30000           0   
             won        65000           0       0        0  65000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley declined        0        0        2           0       0        0  
             pending         0        0        1           2       0        0  
             presented       0    20000        1           0       0        2  
             won             0        0        1           0       0        0

In [22]:
# Or just string the query method to the previous code.
# Pass a dictionary to the aggfunc to perform different functions.
# Each value can have a dictionary. Need to remove totals (why?).
pd.pivot_table(df,
               values=['Price', 'Quantity'],
               index=['Manager', 'Status'],
               columns=['Product'],
               aggfunc={'Quantity':len, 'Price':[np.sum, np.mean]},
               fill_value=0).round(2).query('Manager == ["Debra Henley"]')

Price                                                  \
                         mean                                 sum               
Product                   CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                             
Debra Henley declined   35000           0       0        0  70000           0   
             pending    40000        5000       0        0  40000       10000   
             presented  30000           0       0    10000  30000           0   
             won        65000           0       0        0  65000           0   

                                        Quantity                               
                                             len                               
Product                Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                            
Debra Henley declined        0        0        2           0       0        0  
             pending         0        0        1           2       0        0  
             presented       0    20000        1           0       0        2  
             won             0        0        1           0       0        0

In [23]:
# Another query.
table.query('Status == ["pending", "won"]')

Price                                                   \
                        mean                                  sum               
Product                  CPU Maintenance Monitor Software     CPU Maintenance   
Manager       Status                                                            
Debra Henley  pending  40000        5000       0        0   40000       10000   
              won      65000           0       0        0   65000           0   
Fred Anderson pending      0        5000       0        0       0        5000   
              won      82500        7000       0        0  165000        7000   

                                       Quantity                               
                                            len                               
Product               Monitor Software      CPU Maintenance Monitor Software  
Manager       Status                                                          
Debra Henley  pending       0        0        1           2       0        0  
              won           0        0        1           0       0        0  
Fred Anderson pending       0        0        0           1       0        0  
              won           0        0        2           1       0        0

In [24]:
# And another query.
table.query('Status == ["pending", "won"]').query('Manager == ["Debra Henley"]')

Price                                                  \
                       mean                                 sum               
Product                 CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                           
Debra Henley pending  40000        5000       0        0  40000       10000   
             won      65000           0       0        0  65000           0   

                                      Quantity                               
                                           len                               
Product              Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                          
Debra Henley pending       0        0        1           2       0        0  
             won           0        0        1           0       0        0

In [25]:
# Or this way.
table.query('Status == ["pending", "won"] & Manager == ["Debra Henley"]')

Price                                                  \
                       mean                                 sum               
Product                 CPU Maintenance Monitor Software    CPU Maintenance   
Manager      Status                                                           
Debra Henley pending  40000        5000       0        0  40000       10000   
             won      65000           0       0        0  65000           0   

                                      Quantity                               
                                           len                               
Product              Monitor Software      CPU Maintenance Monitor Software  
Manager      Status                                                          
Debra Henley pending       0        0        1           2       0        0  
             won           0        0        1           0       0        0

# References

- [pandas.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)
- [pandas.query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)